# Pre calculation of Gaunt coefficients

This notebook is used to pre-calculate Gaunt coefficients that are used in the
ViPErLEED Delta-Amplitude calculation. These coefficients are somewhat expensive
to calculate, but can be easily tabulated.

Note, the Gaunt coefficients (also referred to as Wigner 3-j symbols) are
directly related to the Clebsch-Gordan coefficients via mathematical relations
(see also https://en.wikipedia.org/wiki/3-j_symbol). The terms are used
interchangeably in LEED literature.

We use the Sympy physics.wigner module to calculate the values and store them in
a plain-text format.

@author: amimre (Alexander Imre)

In [14]:
import numpy as np
from sympy.physics.wigner import gaunt
from pathlib import Path

In [9]:
# Maximum angular momentum
LMAX = 18

In [10]:
reduced_gaunt_coefficients = np.zeros(
    shape=(LMAX+1, LMAX+1, 2*LMAX+1, 2*LMAX+1, 2*LMAX+1),
    dtype=np.float64
)

We can make use of selection rules for the Gaunt coefficients. Non-zero values
only occur for:

1) $ M1 + M2 + M3 = 0 $
2) $ |L1-L2| <= L3 <= L1+L2 $

The first condition means that we can skip the loop over M3, as only 
$M3 = -M1-M2$ will yield non-zero values. We can skip this dimension entirely
which reduces the memory requirements and file size siginificantly.
The second condition means we can reduce the range for each $L3$
from $(0,2*LMAX+1)$ to $(|L1-L2|, L1+L2)$.

In [11]:
# Calculate the coefficients - may take upwards of 10 minutes for LMAX=18
for l1 in range(LMAX+1):
    for l2 in range(LMAX+1):
        for l3 in range(abs(l1-l2), l1+l2+1):
            for m1 in range(-l1, l1+1):
                for m2 in range(-l2, l2+1):
                    reduced_gaunt_coefficients[l1, l2, l3, m1, m2] = float(gaunt(l1, l2, l3, m1, m2, -m1-m2).doit())

In [12]:
# Size in memory
print(round(reduced_gaunt_coefficients.__sizeof__() / 1024**2, 2), "MB")

139.51 MB


In [16]:
# Save to file
file = Path('.') / 'gaunt_coefficients.npy'
np.save(file, reduced_gaunt_coefficients)

In [17]:
# Size on disk
print(round(file.stat().st_size / 1024**2, 2), "MB")

139.51 MB


In [18]:
# Load from file and check that it is the same
loaded = np.load(file)
print(np.allclose(reduced_gaunt_coefficients, loaded))

True


In [21]:
loaded.dtype

dtype('float64')